Чат, канал менее 200 п и канал с более 200 п

In [1]:
import asyncio
import csv
from telethon import TelegramClient
from telethon.tl.functions.messages import GetHistoryRequest

# Чтение учетных данных из файла
with open("key_tg.txt", "r") as fil:
    api_id = int(fil.readline())  # Ваш API ID
    api_hash = fil.readline()  # Ваш API Hash
    phone = fil.readline()  # Номер телефона, привязанный к аккаунту
chat_link = 'https://t.me/mash_chat'

In [2]:
async def parse_telegram_messages(api_id, api_hash, phone, chat_link, limit=100):
    """
    Парсит сообщения из чата Telegram
    
    :param api_id: Ваш API ID
    :param api_hash: Ваш API Hash
    :param phone: Ваш номер телефона
    :param chat_link: Ссылка на чат (например, 'https://t.me/c/1234567890/123' или '@username')
    :param limit: Количество сообщений для получения
    :return: Список сообщений
    """
    client = TelegramClient(f'session_{phone}', api_id, api_hash)
    
    await client.start(phone=phone)
    
    try:
        # Получаем информацию о чате
        chat_entity = await client.get_entity(chat_link)
        # return client.get_participants(chat_entity, limit=limit)
        # Получаем сообщения
        messages = await client.get_messages(chat_entity, limit=limit)
        
        # Парсим сообщения
        parsed_messages = []
        for message in messages:
            parsed_messages.append({
                'id': message.id,
                'date': message.date,
                'sender_id': message.sender_id,
                'text': message.text,
                'media': message.media
            })
        
        return parsed_messages
        
    finally:
        await client.disconnect()

In [3]:
async def main():
    print(api_id, api_hash)
    messages = await parse_telegram_messages(api_id, api_hash, phone, chat_link, limit=50)
    
    for msg in messages:
        print(f"{msg['date']}: {msg['text']}")
    
    return messages

messages = await main()

27194440 a16d08472f63cc7a196d6d1207ea9bab

2025-10-15 13:14:14+00:00: Девочки , мальчики , всем привет ) Хочу с кем то пообщаться , а может и выйти пройтись, будем смотреть по погоде уже ) Пишите, в общем
2025-10-15 10:31:19+00:00: «Транзакция» и «трансакция» — оба слова происходят от латинского transactio и могут означать сделку или соглашение, но их использование различается в зависимости от контекста. 

Транзакция чаще используется в сфере информатики (последовательность операций с базой данных) и финансов (банковская операция), а трансакция — в политике, юриспруденции и экономике для обозначения более широкого спектра сделок и соглашений
2025-10-15 10:27:59+00:00: https://t.me/mash/68631 

А точно ли «трансакции» а не «транзакции»? Редакторов своих отправьте на курсы русского языка.
2025-10-14 22:36:41+00:00: Не удержался 😂
2025-10-14 19:37:38+00:00: Борисоглебск воронежская область сбивают дроны
2025-10-14 18:06:37+00:00: 
2025-10-14 18:06:37+00:00: Сакральный смысл названия "MASH

In [9]:
from telethon import TelegramClient
from telethon.tl.types import Channel, MessageMediaPhoto, MessageMediaDocument
import asyncio
import json
from datetime import datetime

async def parse_telegram_channel(api_id, api_hash, phone, channel_link, limit=100, parse_media=False):
    """
    Парсит сообщения из канала Telegram
    
    :param api_id: Ваш API ID
    :param api_hash: Ваш API Hash
    :param phone: Ваш номер телефона
    :param channel_link: Ссылка на канал (username или invite link)
    :param limit: Количество сообщений для получения
    :param parse_media: Парсить ли информацию о медиафайлах
    :return: Список сообщений с детальной информацией
    """
    client = TelegramClient(f'session_{phone}', api_id, api_hash)
    
    await client.start(phone=phone)
    
    try:
        # Получаем информацию о канале
        channel_entity = await client.get_entity(channel_link)
        
        # Проверяем, что это действительно канал
        if not isinstance(channel_entity, Channel):
            raise ValueError("Указанная ссылка не ведет к каналу")
        
        print(f"Парсим канал: {channel_entity.title}")
        print(f"Участников: {getattr(channel_entity, 'participants_count', 'N/A')}")
        
        # Получаем сообщения
        messages = await client.get_messages(channel_entity, limit=limit)
        
        # Парсим сообщения
        parsed_messages = []
        for message in messages:
            if not message.message and not parse_media:
                continue  # Пропускаем сообщения без текста, если не парсим медиа
                
            message_data = {
                'id': message.id,
                'date': message.date.isoformat() if message.date else None,
                'text': message.text or '',
                'views': getattr(message, 'views', 0),
                'forwards': getattr(message, 'forwards', 0),
                'replies': getattr(message.replies, 'replies', 0) if message.replies else 0,
                'is_post': getattr(message, 'post', False),
                'channel_id': channel_entity.id,
                'channel_title': channel_entity.title,
                'channel_username': getattr(channel_entity, 'username', None)
            }
            
            # Парсим медиафайлы
            if parse_media and message.media:
                message_data['media'] = parse_media_info(message.media)
            
            # Парсим реакции
            if hasattr(message, 'reactions') and message.reactions:
                message_data['reactions'] = parse_reactions(message.reactions)
            
            parsed_messages.append(message_data)
        
        print(f"Успешно получено {len(parsed_messages)} сообщений")
        return parsed_messages
        
    except Exception as e:
        print(f"Ошибка при парсинге канала: {e}")
        return []
    finally:
        await client.disconnect()

def parse_media_info(media):
    """Парсит информацию о медиафайлах"""
    media_info = {'type': type(media).__name__}
    
    if isinstance(media, MessageMediaPhoto):
        media_info.update({
            'type': 'photo',
            'caption': getattr(media, 'caption', '')
        })
    elif isinstance(media, MessageMediaDocument):
        document = media.document
        media_info.update({
            'type': 'document',
            'mime_type': document.mime_type,
            'size': document.size,
            'caption': getattr(media, 'caption', '')
        })
        # Определяем тип документа
        if document.mime_type:
            if 'image' in document.mime_type:
                media_info['subtype'] = 'image'
            elif 'video' in document.mime_type:
                media_info['subtype'] = 'video'
            elif 'audio' in document.mime_type:
                media_info['subtype'] = 'audio'
    
    return media_info

def parse_reactions(reactions):
    """Парсит информацию о реакциях"""
    reactions_data = {}
    for reaction in reactions.results:
        emoji = getattr(reaction.reaction, 'emoticon', 'custom')
        count = reaction.count
        reactions_data[emoji] = count
    return reactions_data

# Функция для сохранения результатов
def save_messages_to_file(messages, filename=None):
    """Сохраняет сообщения в JSON файл"""
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f'telegram_channel_messages_{timestamp}.json'
    
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(messages, f, ensure_ascii=False, indent=2, default=str)
    
    print(f"Сообщения сохранены в файл: {filename}")

# Использование в Jupyter Notebook
async def main_chan(channel_link):
    # Парсим канал
    messages = await parse_telegram_channel(
        api_id=api_id,
        api_hash=api_hash,
        phone=phone,
        channel_link=channel_link,
        limit=200,  # Количество сообщений
        parse_media=True  # Парсить медиафайлы
    )
    
    # Выводим статистику
    print(f"\n--- Статистика ---")
    print(f"Всего сообщений: {len(messages)}")
    
    messages_with_text = [msg for msg in messages if msg['text']]
    messages_with_media = [msg for msg in messages if msg.get('media')]
    
    print(f"Сообщений с текстом: {len(messages_with_text)}")
    print(f"Сообщений с медиа: {len(messages_with_media)}")
    # print(f"Всего просмотров: {sum(msg['views'] for msg in messages)}")
    # print(f"Всего репостов: {sum(msg['forwards'] for msg in messages)}")
    
    # Сохраняем в файл
    save_messages_to_file(messages)
    
    # Выводим последние 5 сообщений
    print(f"\n--- Последние 5 сообщений ---")
    for msg in messages[:5]:
        print(f"[{msg['date']}] Просмотры: {msg['views']} | {msg['text'][:100]}...")
    
    return messages

In [12]:
messages = await main_chan("https://t.me/fwyllk")

Парсим канал: キラキラ
Участников: None
Успешно получено 200 сообщений

--- Статистика ---
Всего сообщений: 200
Сообщений с текстом: 0
Сообщений с медиа: 200
Сообщения сохранены в файл: telegram_channel_messages_20251015_182429.json

--- Последние 5 сообщений ---
[2025-09-23T19:02:02+00:00] Просмотры: 50 | ...
[2025-09-14T16:03:54+00:00] Просмотры: 94 | ...
[2025-07-31T22:02:36+00:00] Просмотры: 151 | ...
[2025-07-24T06:57:09+00:00] Просмотры: 165 | ...
[2025-07-20T10:35:37+00:00] Просмотры: 188 | ...


In [11]:
messages = await main_chan("https://t.me/cb_economics")

Парсим канал: КБ. экономика
Участников: None
Успешно получено 200 сообщений

--- Статистика ---
Всего сообщений: 200
Сообщений с текстом: 165
Сообщений с медиа: 136
Сообщения сохранены в файл: telegram_channel_messages_20251015_182215.json

--- Последние 5 сообщений ---
[2025-10-15T13:07:04+00:00] Просмотры: 15399 | **Транснефть будет обмениваться информацией с Федеральной службой по аккредитации в режиме реального...
[2025-10-15T12:57:02+00:00] Просмотры: 22116 | Обладатель Нобелевской Премии Мира Корина Мачадо предлагает Дональду Трампу забрать всю нефть Венесу...
[2025-10-15T12:27:55+00:00] Просмотры: 34305 | Психопаты зарабатывают быстрее обычных людей — психологи.

Врачи установили, что нарциссы без совест...
[2025-10-15T11:57:02+00:00] Просмотры: 40765 | В России [могут](https://riamo.ru/news/obschestvo/v-rossii-mogut-vvesti-shtrafy-za-rasprostranenie-f...
[2025-10-15T11:27:02+00:00] Просмотры: 47999 | Обстановка в мире Android - Google Pixel 10 Pro Fold загорелся в руках блогера